In [1]:
import pickle
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import librosa
import numpy as np
import time

In [2]:
test_dir = '/Volumes/FilesHDD/CODE/mmai-proj/nsynth-test/audio/'

In [14]:
def feature_extract(file):
    """
    Define function that takes in a file an returns features in an array
    """
    #print ("Extracting features for {}".format (file[len(test_dir):]))
    
    #get wave representation
    y, sr = librosa.load(file)
        
    #determine if instruemnt is harmonic or percussive by comparing means
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    if np.mean(y_harmonic)>np.mean(y_percussive):
        harmonic=1
    else:
        harmonic=0
        
    #Mel-frequency cepstral coefficients (MFCCs)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    #temporal averaging
    mfcc=np.mean(mfcc,axis=1)
    
    #get the mel-scaled spectrogram
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,fmax=8000)  
    #temporally average spectrogram
    spectrogram = np.mean(spectrogram, axis = 1)
    
    #compute chroma energy
    chroma = librosa.feature.chroma_cens(y=y, sr=sr)
    #temporally average chroma
    chroma = np.mean(chroma, axis = 1)
    
    #compute spectral contrast
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    contrast = np.mean(contrast, axis= 1)
    
    return [harmonic, mfcc, spectrogram, chroma, contrast]

In [15]:
def instrument_code(filename):
    class_names=['bass', 'brass', 'flute', 'guitar', 
             'keyboard', 'mallet', 'organ', 'reed', 
             'string', 'synth_lead', 'vocal']
    
    for name in class_names:
        if name in filename:
            return class_names.index(name)
    else:
        return None

In [16]:
df_test = pd.read_json (path_or_buf='nsynth-test/examples.json', orient='index')
df_test = df_test[df_test['instrument_family']!=9]

In [17]:
filenames_test = df_test.index.tolist()
with open('pickles/filenames_test.pickle', 'wb') as f:
    pickle.dump(filenames_test, f)

In [18]:
start_extract = time.time()
dict_test = {}

for file in filenames_test:
    #extract the features
    features = feature_extract(test_dir + file + '.wav') #specify directory and .wav
    dict_test[file] = features

end_extract = time.time()
print('Time to extract {} files is {} seconds'.format(len(filenames_test), end_extract - start_extract))

Time to extract 4096 files is 959.1106181144714 seconds


In [25]:
print('Converting to dataframe')

#convert dict to dataframe
features_test = pd.DataFrame.from_dict(dict_test, orient='index',
                                       columns=['harmonic', 'mfcc', 'spectro', 'chroma', 'contrast'])

#extract mfccs
mfcc_test = pd.DataFrame(features_test.mfcc.values.tolist(),
                          index=features_test.index)
mfcc_test = mfcc_test.add_prefix('mfcc_')

#extract spectro
spectro_test = pd.DataFrame(features_test.spectro.values.tolist(),
                             index=features_test.index)
spectro_test = spectro_test.add_prefix('spectro_')


#extract chroma
chroma_test = pd.DataFrame(features_test.chroma.values.tolist(),
                            index=features_test.index)
chroma_test = chroma_test.add_prefix('chroma_')


#extract contrast
contrast_test = pd.DataFrame(features_test.contrast.values.tolist(),
                              index=features_test.index)
contrast_test = chroma_test.add_prefix('contrast_')

#drop the old columns
features_test = features_test.drop(labels=['mfcc', 'spectro', 'chroma', 'contrast'], axis=1)

#concatenate
df_features_test=pd.concat([features_test, mfcc_test, spectro_test, chroma_test, contrast_test],
                           axis=1, join='inner')

targets_test = []
for name in df_features_test.index.tolist():
    targets_test.append(instrument_code(name))

Converting to dataframe


In [29]:
#save the dataframe to a pickle file
df_features_test['targets'] = targets_test
with open('pickles/df_features_test.pickle', 'wb') as f:
    pickle.dump(df_features_test, f)
    
df_features_test.shape

(4096, 167)

[0,
 4,
 3,
 7,
 2,
 8,
 10,
 1,
 3,
 8,
 0,
 4,
 4,
 0,
 5,
 3,
 2,
 3,
 6,
 6,
 6,
 0,
 0,
 4,
 3,
 0,
 4,
 3,
 0,
 3,
 5,
 0,
 0,
 2,
 7,
 6,
 6,
 0,
 7,
 3,
 3,
 6,
 0,
 0,
 4,
 4,
 7,
 0,
 2,
 0,
 3,
 0,
 4,
 3,
 3,
 4,
 4,
 6,
 4,
 0,
 6,
 6,
 2,
 3,
 4,
 3,
 0,
 6,
 3,
 5,
 0,
 3,
 4,
 0,
 4,
 6,
 4,
 7,
 4,
 4,
 3,
 8,
 7,
 10,
 0,
 0,
 2,
 0,
 6,
 4,
 0,
 2,
 3,
 6,
 8,
 3,
 0,
 6,
 2,
 4,
 8,
 1,
 5,
 6,
 7,
 6,
 5,
 4,
 6,
 4,
 4,
 2,
 4,
 10,
 3,
 0,
 0,
 6,
 3,
 4,
 2,
 4,
 3,
 1,
 3,
 1,
 4,
 3,
 3,
 4,
 0,
 0,
 6,
 7,
 4,
 1,
 0,
 4,
 4,
 0,
 0,
 6,
 6,
 6,
 3,
 10,
 1,
 10,
 4,
 4,
 4,
 6,
 3,
 6,
 4,
 6,
 4,
 0,
 3,
 4,
 0,
 0,
 1,
 6,
 4,
 3,
 10,
 3,
 0,
 0,
 3,
 1,
 0,
 8,
 3,
 1,
 6,
 5,
 8,
 3,
 1,
 1,
 0,
 4,
 0,
 3,
 3,
 6,
 10,
 8,
 4,
 3,
 6,
 4,
 0,
 7,
 1,
 6,
 3,
 0,
 4,
 5,
 6,
 2,
 8,
 7,
 6,
 3,
 8,
 4,
 6,
 5,
 3,
 4,
 6,
 0,
 4,
 2,
 3,
 1,
 0,
 0,
 4,
 10,
 1,
 6,
 8,
 2,
 4,
 4,
 0,
 7,
 0,
 0,
 7,
 0,
 4,
 3,
 4,
 4,
 4,
 3,
 4,
 0,
 5,
 6,
 3,
 0,
